In [1]:
import os
import time
import numpy as np
import pandas as pd
import audb
import audiofile
import opensmile
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import mutual_info_classif

SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [2]:
smile = opensmile.Smile(feature_set=opensmile.FeatureSet.emobase, feature_level=opensmile.FeatureLevel.Functionals, num_workers=os.cpu_count()-1)
smile.feature_names

['pcm_intensity_sma_max',
 'pcm_intensity_sma_min',
 'pcm_intensity_sma_range',
 'pcm_intensity_sma_maxPos',
 'pcm_intensity_sma_minPos',
 'pcm_intensity_sma_amean',
 'pcm_intensity_sma_linregc1',
 'pcm_intensity_sma_linregc2',
 'pcm_intensity_sma_linregerrA',
 'pcm_intensity_sma_linregerrQ',
 'pcm_intensity_sma_stddev',
 'pcm_intensity_sma_skewness',
 'pcm_intensity_sma_kurtosis',
 'pcm_intensity_sma_quartile1',
 'pcm_intensity_sma_quartile2',
 'pcm_intensity_sma_quartile3',
 'pcm_intensity_sma_iqr1-2',
 'pcm_intensity_sma_iqr2-3',
 'pcm_intensity_sma_iqr1-3',
 'pcm_loudness_sma_max',
 'pcm_loudness_sma_min',
 'pcm_loudness_sma_range',
 'pcm_loudness_sma_maxPos',
 'pcm_loudness_sma_minPos',
 'pcm_loudness_sma_amean',
 'pcm_loudness_sma_linregc1',
 'pcm_loudness_sma_linregc2',
 'pcm_loudness_sma_linregerrA',
 'pcm_loudness_sma_linregerrQ',
 'pcm_loudness_sma_stddev',
 'pcm_loudness_sma_skewness',
 'pcm_loudness_sma_kurtosis',
 'pcm_loudness_sma_quartile1',
 'pcm_loudness_sma_quartile2'

In [3]:
data_dir = os.path.join('emodb\\wav')
file_list = os.listdir(data_dir)
print(file_list)
# all_files = []
# for i in file_list:
#     file_list_path = os.path.join(data_dir,i)
#     file_paths = [os.path.join(file_list_path,j) for j in os.listdir(file_list_path)]
#     all_files.extend(file_paths)
#     # print(os.path.join(data_dir,i,j) for j in file_names)
#     # all_files.append(file_name for file_name in file_names)
# print(all_files)

['03a01Fa.wav', '03a01Nc.wav', '03a01Wa.wav', '03a02Fc.wav', '03a02Nc.wav', '03a02Ta.wav', '03a02Wb.wav', '03a02Wc.wav', '03a04Ad.wav', '03a04Fd.wav', '03a04Lc.wav', '03a04Nc.wav', '03a04Ta.wav', '03a04Wc.wav', '03a05Aa.wav', '03a05Fc.wav', '03a05Nd.wav', '03a05Tc.wav', '03a05Wa.wav', '03a05Wb.wav', '03a07Fa.wav', '03a07Fb.wav', '03a07La.wav', '03a07Nc.wav', '03a07Wc.wav', '03b01Fa.wav', '03b01Lb.wav', '03b01Nb.wav', '03b01Td.wav', '03b01Wa.wav', '03b01Wc.wav', '03b02Aa.wav', '03b02La.wav', '03b02Na.wav', '03b02Tb.wav', '03b02Wb.wav', '03b03Nb.wav', '03b03Tc.wav', '03b03Wc.wav', '03b09La.wav', '03b09Nc.wav', '03b09Tc.wav', '03b09Wa.wav', '03b10Ab.wav', '03b10Ec.wav', '03b10Na.wav', '03b10Nc.wav', '03b10Wb.wav', '03b10Wc.wav', '08a01Ab.wav', '08a01Fd.wav', '08a01Lc.wav', '08a01Na.wav', '08a01Wa.wav', '08a01Wc.wav', '08a02Ab.wav', '08a02Ac.wav', '08a02Fe.wav', '08a02La.wav', '08a02Na.wav', '08a02Tb.wav', '08a02Wc.wav', '08a04Ff.wav', '08a04La.wav', '08a04Nc.wav', '08a04Tb.wav', '08a04Wc.

In [4]:
dir_path = "emodb\\wav"
def gen_file_path(files_names):
    full_paths = [os.path.join(dir_path, i) for i in files_names]
    return full_paths    

In [5]:
dataset_files = gen_file_path(file_list)
dataset_files

['emodb\\wav\\03a01Fa.wav',
 'emodb\\wav\\03a01Nc.wav',
 'emodb\\wav\\03a01Wa.wav',
 'emodb\\wav\\03a02Fc.wav',
 'emodb\\wav\\03a02Nc.wav',
 'emodb\\wav\\03a02Ta.wav',
 'emodb\\wav\\03a02Wb.wav',
 'emodb\\wav\\03a02Wc.wav',
 'emodb\\wav\\03a04Ad.wav',
 'emodb\\wav\\03a04Fd.wav',
 'emodb\\wav\\03a04Lc.wav',
 'emodb\\wav\\03a04Nc.wav',
 'emodb\\wav\\03a04Ta.wav',
 'emodb\\wav\\03a04Wc.wav',
 'emodb\\wav\\03a05Aa.wav',
 'emodb\\wav\\03a05Fc.wav',
 'emodb\\wav\\03a05Nd.wav',
 'emodb\\wav\\03a05Tc.wav',
 'emodb\\wav\\03a05Wa.wav',
 'emodb\\wav\\03a05Wb.wav',
 'emodb\\wav\\03a07Fa.wav',
 'emodb\\wav\\03a07Fb.wav',
 'emodb\\wav\\03a07La.wav',
 'emodb\\wav\\03a07Nc.wav',
 'emodb\\wav\\03a07Wc.wav',
 'emodb\\wav\\03b01Fa.wav',
 'emodb\\wav\\03b01Lb.wav',
 'emodb\\wav\\03b01Nb.wav',
 'emodb\\wav\\03b01Td.wav',
 'emodb\\wav\\03b01Wa.wav',
 'emodb\\wav\\03b01Wc.wav',
 'emodb\\wav\\03b02Aa.wav',
 'emodb\\wav\\03b02La.wav',
 'emodb\\wav\\03b02Na.wav',
 'emodb\\wav\\03b02Tb.wav',
 'emodb\\wav\\03b02W

In [7]:
df = smile.process_files(dataset_files)
df

,,,pcm_intensity_sma_max,pcm_intensity_sma_min,pcm_intensity_sma_range,pcm_intensity_sma_maxPos,pcm_intensity_sma_minPos,pcm_intensity_sma_amean,pcm_intensity_sma_linregc1,pcm_intensity_sma_linregc2,pcm_intensity_sma_linregerrA,pcm_intensity_sma_linregerrQ,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
file,start,end,,,,,,,,,,,,,,,,,,,,,
emodb\wav\03a01Fa.wav,0 days,0 days 00:00:01.898250,0.000085,9.283679e-10,0.000085,16.0,8.0,0.000005,-6.005426e-08,0.000010,0.000006,1.079555e-10,...,60.561657,7.811279,1.506308,7.152560,-0.595657,0.0,2.998678,0.595657,2.998678,3.594335
emodb\wav\03a01Nc.wav,0 days,0 days 00:00:01.611250,0.000085,6.605034e-11,0.000085,67.0,137.0,0.000010,-1.164632e-07,0.000019,0.000010,2.380510e-10,...,10.288302,3.331336,1.420164,6.868492,-0.110564,0.0,1.374932,0.110564,1.374932,1.485495
emodb\wav\03a01Wa.wav,0 days,0 days 00:00:01.877812500,0.000224,2.558865e-11,0.000224,36.0,95.0,0.000016,-6.710033e-08,0.000022,0.000021,1.361215e-09,...,33.031918,6.118035,1.905320,12.911694,-0.452303,0.0,2.030742,0.452303,2.030742,2.483046
emodb\wav\03a02Fc.wav,0 days,0 days 00:00:02.006250,0.000152,8.704463e-11,0.000152,106.0,7.0,0.000010,-3.877498e-08,0.000014,0.000012,4.737973e-10,...,99.266121,10.403279,3.130466,20.958641,-2.255981,0.0,2.846315,2.255981,2.846315,5.102296
emodb\wav\03a02Nc.wav,0 days,0 days 00:00:01.439812500,0.000190,3.851465e-10,0.000190,45.0,0.0,0.000017,-1.658279e-07,0.000028,0.000021,1.211545e-09,...,98.266998,9.982560,3.386227,18.761950,-1.456049,0.0,1.939871,1.456049,1.939871,3.395920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
emodb\wav\16b10Lb.wav,0 days,0 days 00:00:03.442687500,0.000191,1.620664e-09,0.000191,134.0,12.0,0.000014,-8.838911e-08,0.000029,0.000016,7.963888e-10,...,59.050297,7.688866,3.757301,26.560385,-1.617197,0.0,1.727139,1.617197,1.727139,3.344337
emodb\wav\16b10Tb.wav,0 days,0 days 00:00:03.500625,0.000251,8.674476e-10,0.000251,131.0,188.0,0.000026,-3.238233e-08,0.000032,0.000032,2.093426e-09,...,26.929825,5.205812,1.002882,5.362481,-1.323231,0.0,2.483423,1.323231,2.483423,3.806654
emodb\wav\16b10Td.wav,0 days,0 days 00:00:03.934187500,0.000096,1.877495e-09,0.000096,17.0,259.0,0.000010,-5.030551e-08,0.000020,0.000012,2.993261e-10,...,103.201447,10.260765,2.094101,15.084125,-3.998133,0.0,3.901196,3.998133,3.901196,7.899329


In [8]:
df.to_csv(path_or_buf='datasetEmo.csv')

In [2]:
data = pd.read_csv('datasetEmo.csv')

anger       W <br>
boredom	    L <br>
disgust	    E <br>
fear        A <br>
happiness	F <br>
sadness	    T <br>
neutral     N <br>

In [3]:
def get_file_emotion(file_path):
    # file_path = "data\\AudioData\\KL\\n17.wav"
    file_parts = file_path.split("\\")
    file_name = file_parts[-1]
    file_name = file_name.split('.')[0]
    file_emotion = file_name[-2]
    if file_emotion == "W":
        return 'anger'
    elif file_emotion == "L":
        return "boredom"
    elif file_emotion == "E":
        return "disgust"
    elif file_emotion == "A":
        return "fear"
    elif file_emotion == "F":
        return "happiness"
    elif file_emotion == "T":
        return "sadness"
    else:
        return "neutral"

In [4]:
data['emotion'] = data['file'].apply(get_file_emotion)
data

,file,start,end,pcm_intensity_sma_max,pcm_intensity_sma_min,pcm_intensity_sma_range,pcm_intensity_sma_maxPos,pcm_intensity_sma_minPos,pcm_intensity_sma_amean,pcm_intensity_sma_linregc1,...,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3,emotion
0,emodb\wav\03a01Fa.wav,0 days,0 days 00:00:01.898250,0.000085,9.283679e-10,0.000085,16.0,8.0,0.000005,-6.005426e-08,...,7.811279,1.506308,7.152560,-0.595657,0.0,2.998678,0.595657,2.998678,3.594335,happiness
1,emodb\wav\03a01Nc.wav,0 days,0 days 00:00:01.611250,0.000085,6.605034e-11,0.000085,67.0,137.0,0.000010,-1.164632e-07,...,3.331335,1.420164,6.868492,-0.110564,0.0,1.374932,0.110564,1.374932,1.485495,neutral
2,emodb\wav\03a01Wa.wav,0 days,0 days 00:00:01.877812500,0.000224,2.558865e-11,0.000224,36.0,95.0,0.000016,-6.710033e-08,...,6.118035,1.905320,12.911694,-0.452303,0.0,2.030742,0.452303,2.030742,2.483046,anger
3,emodb\wav\03a02Fc.wav,0 days,0 days 00:00:02.006250,0.000152,8.704463e-11,0.000152,106.0,7.0,0.000010,-3.877498e-08,...,10.403279,3.130466,20.958641,-2.255981,0.0,2.846315,2.255981,2.846315,5.102296,happiness
4,emodb\wav\03a02Nc.wav,0 days,0 days 00:00:01.439812500,0.000190,3.851465e-10,0.000190,45.0,0.0,0.000017,-1.658279e-07,...,9.982560,3.386227,18.761950,-1.456049,0.0,1.939871,1.456049,1.939871,3.395920,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,emodb\wav\16b10Lb.wav,0 days,0 days 00:00:03.442687500,0.000191,1.620664e-09,0.000191,134.0,12.0,0.000014,-8.838912e-08,...,7.688866,3.757301,26.560385,-1.617197,0.0,1.727139,1.617197,1.727139,3.344336,boredom
531,emodb\wav\16b10Tb.wav,0 days,0 days 00:00:03.500625,0.000251,8.674476e-10,0.000251,131.0,188.0,0.000026,-3.238233e-08,...,5.205813,1.002881,5.362481,-1.323231,0.0,2.483423,1.323231,2.483423,3.806654,sadness
532,emodb\wav\16b10Td.wav,0 days,0 days 00:00:03.934187500,0.000096,1.877495e-09,0.000096,17.0,259.0,0.000010,-5.030551e-08,...,10.260765,2.094101,15.084125,-3.998133,0.0,3.901196,3.998133,3.901196,7.899329,sadness
533,emodb\wav\16b10Wa.wav,0 days,0 days 00:00:02.414125,0.000124,1.844543e-11,0.000124,99.0,10.0,0.000003,2.645785e-09,...,9.361651,3.588442,24.238676,-2.969704,0.0,2.841788,2.969704,2.841788,5.811492,anger


In [5]:
data = data.drop(['file', 'start', 'end'], axis='columns')
data

,pcm_intensity_sma_max,pcm_intensity_sma_min,pcm_intensity_sma_range,pcm_intensity_sma_maxPos,pcm_intensity_sma_minPos,pcm_intensity_sma_amean,pcm_intensity_sma_linregc1,pcm_intensity_sma_linregc2,pcm_intensity_sma_linregerrA,pcm_intensity_sma_linregerrQ,...,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3,emotion
0,0.000085,9.283679e-10,0.000085,16.0,8.0,0.000005,-6.005426e-08,0.000010,0.000006,1.079555e-10,...,7.811279,1.506308,7.152560,-0.595657,0.0,2.998678,0.595657,2.998678,3.594335,happiness
1,0.000085,6.605034e-11,0.000085,67.0,137.0,0.000010,-1.164632e-07,0.000019,0.000010,2.380510e-10,...,3.331335,1.420164,6.868492,-0.110564,0.0,1.374932,0.110564,1.374932,1.485495,neutral
2,0.000224,2.558865e-11,0.000224,36.0,95.0,0.000016,-6.710033e-08,0.000022,0.000021,1.361215e-09,...,6.118035,1.905320,12.911694,-0.452303,0.0,2.030742,0.452303,2.030742,2.483046,anger
3,0.000152,8.704463e-11,0.000152,106.0,7.0,0.000010,-3.877498e-08,0.000014,0.000012,4.737973e-10,...,10.403279,3.130466,20.958641,-2.255981,0.0,2.846315,2.255981,2.846315,5.102296,happiness
4,0.000190,3.851465e-10,0.000190,45.0,0.0,0.000017,-1.658279e-07,0.000028,0.000021,1.211545e-09,...,9.982560,3.386227,18.761950,-1.456049,0.0,1.939871,1.456049,1.939871,3.395920,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,0.000191,1.620664e-09,0.000191,134.0,12.0,0.000014,-8.838912e-08,0.000029,0.000016,7.963888e-10,...,7.688866,3.757301,26.560385,-1.617197,0.0,1.727139,1.617197,1.727139,3.344336,boredom
531,0.000251,8.674476e-10,0.000251,131.0,188.0,0.000026,-3.238233e-08,0.000032,0.000032,2.093426e-09,...,5.205813,1.002881,5.362481,-1.323231,0.0,2.483423,1.323231,2.483423,3.806654,sadness
532,0.000096,1.877495e-09,0.000096,17.0,259.0,0.000010,-5.030551e-08,0.000020,0.000012,2.993261e-10,...,10.260765,2.094101,15.084125,-3.998133,0.0,3.901196,3.998133,3.901196,7.899329,sadness
533,0.000124,1.844543e-11,0.000124,99.0,10.0,0.000003,2.645785e-09,0.000003,0.000005,2.038378e-10,...,9.361651,3.588442,24.238676,-2.969704,0.0,2.841788,2.969704,2.841788,5.811492,anger


In [6]:
Y = data['emotion'].to_frame()
Y

,emotion
0,happiness
1,neutral
2,anger
3,happiness
4,neutral
...,...
530,boredom
531,sadness
532,sadness
533,anger


In [7]:
X = data.drop(columns=['emotion'])

In [8]:
norm = MinMaxScaler(feature_range=(-1, 1))
X_norm = norm.fit_transform(X)
X_norm

array([[-7.41440950e-01, -9.96626221e-01, -7.41429083e-01, ...,
        -7.99412590e-01,  1.00230884e-02, -3.08106270e-01],
       [-7.39344890e-01, -9.99759967e-01, -7.39329033e-01, ...,
        -9.62767703e-01, -5.36891636e-01, -7.14048648e-01],
       [-1.17262557e-01, -9.99907008e-01, -1.17215821e-01, ...,
        -8.47686922e-01, -3.15999748e-01, -5.22024607e-01],
       ...,
       [-6.90968555e-01, -9.93177001e-01, -6.90958420e-01, ...,
         3.46370784e-01,  3.14011786e-01,  5.20586203e-01],
       [-5.63519533e-01, -9.99932968e-01, -5.63494807e-01, ...,
         4.72965174e-05, -4.28208707e-02,  1.18686733e-01],
       [-7.84992943e-01, -9.99958117e-01, -7.84979145e-01, ...,
        -1.24780218e-01,  3.77131551e-02,  9.33573902e-02]])

In [9]:
threshold = 5
high_score_feats = []
y = np.ravel(Y.values)
feature_scores = mutual_info_classif(X_norm, y)

In [10]:
feature_scores

array([1.74980815e-02, 3.60201947e-01, 1.78160589e-02, 1.61099044e-01,
       3.80246586e-02, 1.02115958e-01, 2.65618449e-01, 2.87276088e-01,
       4.00808257e-02, 0.00000000e+00, 6.09450099e-02, 1.21905865e-02,
       7.72685835e-02, 1.72389380e-01, 2.21193918e-01, 1.82000254e-01,
       2.39297983e-01, 1.35001462e-01, 1.52139434e-01, 7.15497006e-02,
       3.83204926e-01, 3.46401840e-02, 1.57709640e-01, 5.32113871e-02,
       1.98406944e-01, 2.31552440e-01, 2.17553244e-01, 1.22154259e-01,
       6.14694001e-02, 4.19636835e-02, 1.37533594e-01, 1.02770750e-01,
       1.44326456e-01, 1.71298316e-01, 1.63103711e-01, 9.01863029e-02,
       6.93742949e-02, 6.78286195e-02, 1.48648612e-01, 1.25297736e-01,
       8.18716418e-02, 6.78425777e-02, 8.97212190e-02, 3.94487357e-01,
       2.05306830e-02, 1.97391970e-01, 1.53476092e-01, 8.45173888e-02,
       1.18718614e-01, 1.77883943e-01, 4.72265856e-02, 3.44204116e-01,
       4.50159515e-01, 4.38379069e-01, 2.24927417e-01, 5.52144462e-02,
      

In [11]:
for score, f_name in sorted(zip(feature_scores, X.columns), reverse=True):
        print(f_name, score)
        if score > 0.0:
                high_score_feats.append(f_name)
high_score_feats

pcm_zcr_sma_quartile2 0.6103546235173192
pcm_zcr_sma_quartile1 0.56261902605323
mfcc_sma_de[12]_stddev 0.5253780051103982
mfcc_sma_de[12]_linregerrQ 0.48842673340335896
mfcc_sma_de[2]_linregerrQ 0.4705710593524324
mfcc_sma[2]_quartile1 0.4705591941769067
mfcc_sma_de[2]_stddev 0.4702415581446271
mfcc_sma_de[2]_linregerrA 0.4603308706770144
mfcc_sma[2]_amean 0.4565751742115145
pcm_zcr_sma_de_iqr1-3 0.4537841041230557
mfcc_sma[1]_quartile2 0.4501595149717823
pcm_zcr_sma_de_iqr1-2 0.44595875529289186
mfcc_sma_de[12]_linregerrA 0.4439669028873241
mfcc_sma[2]_quartile2 0.44310032818755873
pcm_zcr_sma_de_iqr2-3 0.4403702047819795
mfcc_sma[1]_quartile3 0.4383790689967477
mfcc_sma[2]_min 0.43774094472029335
mfcc_sma[2]_quartile3 0.4258636973105845
pcm_zcr_sma_iqr1-2 0.4170256010419453
pcm_zcr_sma_de_quartile3 0.41561727333373044
pcm_zcr_sma_de_quartile1 0.40101039291305485
mfcc_sma[1]_amean 0.39448735674133406
mfcc_sma_de[12]_iqr1-3 0.39422440541555526
lspFreq_sma[0]_quartile1 0.391906528292028

['pcm_zcr_sma_quartile2',
 'pcm_zcr_sma_quartile1',
 'mfcc_sma_de[12]_stddev',
 'mfcc_sma_de[12]_linregerrQ',
 'mfcc_sma_de[2]_linregerrQ',
 'mfcc_sma[2]_quartile1',
 'mfcc_sma_de[2]_stddev',
 'mfcc_sma_de[2]_linregerrA',
 'mfcc_sma[2]_amean',
 'pcm_zcr_sma_de_iqr1-3',
 'mfcc_sma[1]_quartile2',
 'pcm_zcr_sma_de_iqr1-2',
 'mfcc_sma_de[12]_linregerrA',
 'mfcc_sma[2]_quartile2',
 'pcm_zcr_sma_de_iqr2-3',
 'mfcc_sma[1]_quartile3',
 'mfcc_sma[2]_min',
 'mfcc_sma[2]_quartile3',
 'pcm_zcr_sma_iqr1-2',
 'pcm_zcr_sma_de_quartile3',
 'pcm_zcr_sma_de_quartile1',
 'mfcc_sma[1]_amean',
 'mfcc_sma_de[12]_iqr1-3',
 'lspFreq_sma[0]_quartile1',
 'mfcc_sma_de[2]_quartile3',
 'mfcc_sma_de[2]_iqr1-2',
 'mfcc_sma[2]_stddev',
 'mfcc_sma_de[5]_linregerrA',
 'pcm_loudness_sma_min',
 'lspFreq_sma[0]_quartile2',
 'mfcc_sma_de[8]_stddev',
 'mfcc_sma_de[5]_linregerrQ',
 'mfcc_sma[2]_iqr1-3',
 'pcm_zcr_sma_quartile3',
 'mfcc_sma_de[9]_stddev',
 'pcm_intensity_sma_min',
 'pcm_zcr_sma_amean',
 'mfcc_sma_de[5]_stddev

In [12]:
X_norm_df = pd.DataFrame(X_norm, columns=X.columns)

In [13]:
data_mic = X_norm_df[high_score_feats]

In [14]:
Y = pd.get_dummies(Y)
Y

,emotion_anger,emotion_boredom,emotion_disgust,emotion_fear,emotion_happiness,emotion_neutral,emotion_sadness
0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0
3,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...
530,0,1,0,0,0,0,0
531,0,0,0,0,0,0,1
532,0,0,0,0,0,0,1
533,1,0,0,0,0,0,0


In [15]:
X_train, X_test, y_train, y_test = train_test_split(data_mic, Y, random_state=27)

In [16]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [17]:
knn.score(X_train, y_train)

0.8354114713216958

In [18]:
knn.score(X_test, y_test)

0.7238805970149254

In [19]:
y_pred = knn.predict(X_test)

In [20]:
print(y_test)

     emotion_anger  emotion_boredom  emotion_disgust  emotion_fear  \
146              1                0                0             0   
211              0                1                0             0   
174              0                1                0             0   
239              0                1                0             0   
448              0                1                0             0   
..             ...              ...              ...           ...   
245              0                0                0             0   
271              0                0                0             0   
383              1                0                0             0   
367              0                0                0             0   
400              1                0                0             0   

     emotion_happiness  emotion_neutral  emotion_sadness  
146                  0                0                0  
211                  0                0  

In [21]:
print(confusion_matrix(y_test.values.argmax(axis=1), y_pred.argmax(axis=1)))

[[32  0  0  0  2  0  0]
 [ 3  9  0  0  0  2  0]
 [ 2  0  7  0  0  0  0]
 [ 1  0  0 12  0  0  0]
 [ 9  0  0  2 10  0  0]
 [ 4  4  0  1  0 17  0]
 [ 2  2  0  0  0  0 13]]


In [22]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.85      0.87        34
           1       0.60      0.64      0.62        14
           2       1.00      0.78      0.88         9
           3       0.80      0.92      0.86        13
           4       0.83      0.48      0.61        21
           5       0.89      0.65      0.76        26
           6       1.00      0.76      0.87        17

   micro avg       0.85      0.72      0.78       134
   macro avg       0.86      0.73      0.78       134
weighted avg       0.86      0.72      0.78       134
 samples avg       0.72      0.72      0.72       134



f:\Virang ka Kaam\Automated-EQ\env\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
print(y_pred)

[[1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 0 0 

In [24]:
knn_cv = KNeighborsClassifier(n_neighbors=3)

In [25]:
cv_scores = cross_val_score(knn_cv, data_mic, Y, cv=20)

In [26]:
print(cv_scores)
print('cv_scores mean:{}'.format(np.mean(cv_scores)))

[0.59259259 0.62962963 0.7037037  0.85185185 0.66666667 0.74074074
 0.66666667 0.66666667 0.55555556 0.7037037  0.77777778 0.59259259
 0.7037037  0.66666667 0.77777778 0.80769231 0.69230769 0.73076923
 0.84615385 0.76923077]
cv_scores mean:0.7071225071225071
